In [ ]:
import numpy as np
import xfab.tools
import os
import numpy as np
import matplotlib.pyplot as plt
from xfab.symmetry import Umis
import xfab.symmetry
from collections import deque
from scipy.ndimage import binary_fill_holes
from scipy.spatial.transform import Rotation
import matplotlib
import torch
import skimage
import scipy.ndimage

In [ ]:
keys = ['layer_top', 'layer_center', 'layer_bottom']
dirs = [os.path.join('..', dir, 'test_run') for dir in keys]
sample = {}
sample['layer_top']    = np.fliplr(  np.load(os.path.join(dirs[0], 'sample_mask_layer_top.npy'))[5:-5, 5:-5]  )
sample['layer_center'] = np.fliplr(  np.load(os.path.join(dirs[1], 'sample_mask_layer_center.npy'))[2:-2, 2:-2] )
sample['layer_bottom'] = np.fliplr(  np.load(os.path.join(dirs[2], 'sample_mask_layer_bottom.npy'))[2:-2, 2:-2] )

stress = {}
stress['layer_top'] = np.load('../layer_top/test_run/stress_map_top_layer_refined_com_corrected.npy')
stress['layer_center'] = np.load('../layer_center/test_run/stress_map_central_layer_refined_com_corrected.npy')
stress['layer_bottom'] = np.load('../layer_bottom/test_run/stress_map_bottom_layer_refined_com_corrected.npy')

strain = {}
strain['layer_top'] = np.load('../layer_top/test_run/strain_map_top_layer_refined_com_corrected.npy')
strain['layer_center'] = np.load('../layer_center/test_run/strain_map_central_layer_refined_com_corrected.npy')
strain['layer_bottom'] = np.load('../layer_bottom/test_run/strain_map_bottom_layer_refined_com_corrected.npy')

ubi_map, X, Y, ipf_map, xypf_map = {},{},{},{}, {}
for i,key in enumerate(keys):
    X[key] = np.load( os.path.join( dirs[i], 'X_coord_gmap.npy' ) )
    Y[key] = np.load( os.path.join( dirs[i], 'Y_coord_gmap.npy' ) )
    ubi_map[key]    = np.load(os.path.join(dirs[i], 'ubi_median_filt_s6.npy'))
    ipf_map[key]    = np.load(os.path.join(dirs[i], 'ipf_median_filt_s6.npy'))
    xypf_map[key]    = np.load(os.path.join(dirs[i], 'xypf_median_filt_s6.npy'))

n = np.max( stress['layer_top'].shape + stress['layer_center'].shape + stress['layer_bottom'].shape )
stress_3D = np.zeros( (3, n,n,3,3) )
for i,key in enumerate(keys):
    k = stress[key].shape[0]
    pad = (n-k)//2
    if pad==0: stress_3D[i] = stress[key]
    else:stress_3D[ i, pad:-pad, pad:-pad] = stress[key]

grain_boundaries = {}
for i,key in enumerate(keys):
    grain_boundaries[key] = np.load(key + '_grain_boundaries.npy')

In [ ]:
strain[key][sample[key]].max(), strain[key][sample[key]].min(), strain[key][sample[key]].std()

In [ ]:

components = [[0,0],[1,1],[2,2],[0,1],[0,2],[1,2]]
titles= 'xx yy zz xy xz yz'.split(' ')
for key in keys:

    for component,title in zip(components, titles):
        fig, ax = plt.subplots(1,1,figsize=(9,9))
        coords = np.argwhere(sample[key])
        x_min, y_min = coords.min(axis=0)
        x_max, y_max = coords.max(axis=0)

        gb = grain_boundaries[key][x_min:x_max+1, y_min:y_max+1].copy()

        cropped = strain[key][x_min:x_max+1, y_min:y_max+1].copy()
            
        Xc = X[key][x_min:x_max+1, y_min:y_max+1]
        Yc = Y[key][x_min:x_max+1, y_min:y_max+1]

        cmap = matplotlib.colormaps.get_cmap('RdBu_r')

        mask = sample[key].copy()[x_min:x_max+1, y_min:y_max+1]
        rgb_arr = cropped[:,:,component[0],component[1]]

        if title=='xx' or title=='yy' or title=='zz':
            vmax = 0.0017
        else:
            vmax = 0.0007

        vmin = -vmax 

        norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)

        rgb_image = cmap( norm(rgb_arr) )
        rgb_image[gb, 0:3] = 0

        rgb_image[~mask, -1] = 0

        im = ax.pcolormesh( Xc, Yc, rgb_image )

        #cax = fig.add_axes([0.95, 0.14, 0.035, 0.7])

        bar = np.linspace(vmax, vmin, 500)
        #cax.imshow( bar.reshape(500,1) * np.ones( (500, 10) ), cmap='RdBu_r' )

        positions = np.linspace(0, len(bar)-1, 9).astype(int)
        labels = [np.round(bar[i]).astype(int) for i in positions]

        #cax.set_yticks(positions, labels)
        #cax.set_xticks([], [])
        #cax.yaxis.tick_right() 

        #cax.spines[['right', 'top', 'left', 'bottom']].set_visible(False)
        #cax.tick_params(axis='both', which='major', labelsize=12)

        ax.axis('off')

        #ax.annotate( '[MPa]', (Xc[-1,-1] ,  Yc[-10,-10]), size=16 )

        #ax.set_title(key.replace('_', ' ') + ' - $\epsilon_{'+title+'}$', fontsize=24)

        fig.savefig('gallery/' + key +'_strain_'+title+'.png', dpi=400, pad_inches=0.000001, bbox_inches='tight')
        plt.show()

In [ ]:

components = [[0,0],[1,1],[2,2],[0,1],[0,2],[1,2]]
titles= 'xx yy zz xy xz yz'.split(' ')
for key in keys:

    for component,title in zip(components, titles):
        fig, ax = plt.subplots(1,1,figsize=(9,9))
        coords = np.argwhere(sample[key])
        x_min, y_min = coords.min(axis=0)
        x_max, y_max = coords.max(axis=0)

        gb = grain_boundaries[key][x_min:x_max+1, y_min:y_max+1].copy()

        cropped = stress[key][x_min:x_max+1, y_min:y_max+1].copy() / 1e6
            
        Xc = X[key][x_min:x_max+1, y_min:y_max+1]
        Yc = Y[key][x_min:x_max+1, y_min:y_max+1]

        cmap = matplotlib.colormaps.get_cmap('RdBu_r')

        mask = sample[key].copy()[x_min:x_max+1, y_min:y_max+1]
        rgb_arr = cropped[:,:,component[0],component[1]]

        if title=='xx' or title=='yy' or title=='zz':
            vmax = 170
        else:
            vmax = 40

        vmin = -vmax 

        norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)

        rgb_image = cmap( norm(rgb_arr) )
        rgb_image[gb, 0:3] = 0

        rgb_image[~mask, -1] = 0

        im = ax.pcolormesh( Xc, Yc, rgb_image )

        #cax = fig.add_axes([0.95, 0.14, 0.035, 0.7])

        bar = np.linspace(vmax, vmin, 500)
        #cax.imshow( bar.reshape(500,1) * np.ones( (500, 10) ), cmap='RdBu_r' )

        positions = np.linspace(0, len(bar)-1, 9).astype(int)
        labels = [np.round(bar[i]).astype(int) for i in positions]

        #cax.set_yticks(positions, labels)
        #cax.set_xticks([], [])
        #cax.yaxis.tick_right() 

        #cax.spines[['right', 'top', 'left', 'bottom']].set_visible(False)
        #cax.tick_params(axis='both', which='major', labelsize=12)

        ax.axis('off')

        #ax.annotate( '[MPa]', (Xc[-1,-1] ,  Yc[-10,-10]), size=16 )

        #ax.set_title(key.replace('_', ' ') + ' - $\epsilon_{'+title+'}$', fontsize=24)

        fig.savefig('gallery/' + key +'_stress_'+title+'.png', dpi=400, pad_inches=0.000001, bbox_inches='tight')
        plt.show()